As the this RF model use the iterative forecasting, the features (independent variable) of the rainfall need to be known before make the next day prediction.

This can be done by make different predictor of each of the features. A few features can use same predictor in order save cost.


| **Features** | **Sources** | **Model** |
|---|---|---| 
| TOTAL, ClimAdjust, ANOM | ENSO | Multiple Output Linear Regression |
| Temperature, DewPoint, Humidity, Visibility, WindSpeed, Pressure | WunderGround | Multiple Output Linear Regression |

#

Import Data and Dependacies

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error


df = pd.read_csv('lawas/lawas-rainfall-daily.csv', encoding='latin')
df["DateTime"] = pd.to_datetime(df["DateTime"],format="%Y-%m-%d")
df.set_index("DateTime",inplace=True)

ENSO: ONI Index (Multiple Output Linear Regression)

In [2]:
# rainfall_feat = df.iloc[: , 1:4]
rainfall_feat = df.iloc[: , 1:10]
print(rainfall_feat)

X = rainfall_feat[:-1].values  # All rows except the last
y = rainfall_feat[1:].values   # All rows except the first

# scaler_X = StandardScaler()
# scaler_y = StandardScaler()

# X_scaled = scaler_X.fit_transform(X)
# y_scaled = scaler_y.fit_transform(y)

mlr = MultiOutputRegressor(LinearRegression())
# mlr = MultiOutputRegressor(Ridge(alpha=0.1))
# mlr = MultiOutputRegressor(RandomForestRegressor())
mlr.fit(X, y)

# Predict the next values for each column
last_row = rainfall_feat.iloc[-1].values.reshape(1, -1)  # The last row of the dataset
predicted_next_values = mlr.predict(last_row)
# predicted_next_values = scaler_y.inverse_transform(predicted_next_values)

            TOTAL  ClimAdjust  ANOM  Temperature   DewPoint   Humidity  \
DateTime                                                                 
2002-08-01  27.72       26.91  0.81    83.500000  73.500000  73.208333   
2002-08-02  27.72       26.91  0.81    82.666667  75.416667  80.375000   
2002-08-03  27.72       26.91  0.81    81.958333  74.250000  78.833333   
2002-08-04  27.72       26.91  0.81    82.791667  73.000000  73.458333   
2002-08-05  27.72       26.91  0.81    81.708333  74.125000  79.500000   
...           ...         ...   ...          ...        ...        ...   
2021-01-30  25.56       26.55 -0.99    81.041667  74.250000  81.125000   
2021-01-31  25.56       26.55 -0.99    81.750000  74.500000  80.041667   
2021-02-01  25.76       26.76 -1.00    81.333333  75.166667  82.708333   
2021-02-02  25.76       26.76 -1.00    80.458333  74.666667  83.833333   
2021-02-03  25.76       26.76 -1.00    79.846154  74.230769  84.000000   

            Visibility  WindSpeed   P

In [3]:
# Display the results
predicted_next_values = np.round(predicted_next_values.flatten(), 2)
print("Predicted next values for each column:")
for column, value in zip(rainfall_feat.columns, predicted_next_values):
    print(f"{column}: {value}")

Predicted next values for each column:
TOTAL: 25.76
ClimAdjust: 26.76
ANOM: -1.0
Temperature: 80.43
DewPoint: 74.27
Humidity: 82.72
Visibility: 6.08
WindSpeed: 7.14
Pressure: 29.71


In [4]:
with open("rainfall_features_predictor.pkl", 'wb') as file:  
    pickle.dump(mlr, file)

In [76]:
# import pandas as pd
# import numpy as np
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# # Read and preprocess data
# df = pd.read_csv('lawas/lawas-rainfall-daily.csv', encoding='latin')
# df["DateTime"] = pd.to_datetime(df["DateTime"], format="%Y-%m-%d")
# df.set_index("DateTime", inplace=True)
# rainfall_feat = df.iloc[:, 1:10]

# # Create time-based features
# rainfall_feat['month'] = rainfall_feat.index.month
# rainfall_feat['day'] = rainfall_feat.index.day
# rainfall_feat['day_of_week'] = rainfall_feat.index.dayofweek

# # Create lagged features (t-1, t-2, t-3)
# for col in ['TOTAL', 'ClimAdjust', 'ANOM', 'Temperature', 'DewPoint', 
#             'Humidity', 'Visibility', 'WindSpeed', 'Pressure']:
#     for lag in range(1, 8):
#         rainfall_feat[f'{col}_lag_{lag}'] = rainfall_feat[col].shift(lag)

# # Drop rows with NaN values from the lagged features
# rainfall_feat = rainfall_feat.dropna()

# # Separate features and target variables
# target_columns = ['TOTAL', 'ClimAdjust', 'ANOM', 'Temperature', 'DewPoint', 
#                  'Humidity', 'Visibility', 'WindSpeed', 'Pressure']
# feature_columns = [col for col in rainfall_feat.columns if col not in target_columns]

# X = rainfall_feat[feature_columns].values
# y = rainfall_feat[target_columns].values

# # Scale the features and target variables
# scaler_X = RobustScaler()
# scaler_y = RobustScaler()

# X_scaled = scaler_X.fit_transform(X)
# y_scaled = scaler_y.fit_transform(y)

# # Create and train the multioutput regression model
# mlr_enso = MultiOutputRegressor(LinearRegression())
# mlr_enso.fit(X_scaled, y_scaled)

# # Function to make predictions
# def predict_next_values(input_data):
#     """
#     Make predictions using the trained model
    
#     Parameters:
#     input_data: DataFrame with the same features as training data
    
#     Returns:
#     DataFrame with predicted values for all target variables
#     """
#     # Prepare input data
#     input_scaled = scaler_X.transform(input_data)
    
#     # Make predictions
#     pred_scaled = mlr_enso.predict(input_scaled)
    
#     # Inverse transform predictions
#     predictions = scaler_y.inverse_transform(pred_scaled)
    
#     # Convert to DataFrame
#     pred_df = pd.DataFrame(predictions, 
#                           columns=target_columns,
#                           index=input_data.index)
#     return pred_df

# # Example of how to make predictions for the next day
# def prepare_next_day_features(rainfall_feat):
#     """
#     Prepare features for next day prediction
#     """
#     latest_data = rainfall_feat.iloc[-1:].copy()
    
#     # Update time features
#     next_day = latest_data.index[0] + pd.Timedelta(days=1)
#     latest_data.index = [next_day]
#     latest_data['month'] = next_day.month
#     latest_data['day'] = next_day.day
#     latest_data['day_of_week'] = next_day.dayofweek
    
#     # Update lag features
#     for col in target_columns:
#         for lag in range(1, 8):
#             latest_data[f'{col}_lag_{lag}'] = rainfall_feat[col].iloc[-(lag+1)]
    
#     return latest_data[feature_columns]

In [77]:
# # Prepare features for next day
# next_day_features = prepare_next_day_features(rainfall_feat)

# # Make predictions
# predictions = predict_next_values(next_day_features)
# print(predictions)

                TOTAL  ClimAdjust      ANOM  Temperature   DewPoint  \
2021-02-04  25.767921   26.767526 -0.999546    80.639337  74.623697   

             Humidity  Visibility  WindSpeed   Pressure  
2021-02-04  83.115706    5.766653   6.550918  29.727203  


d:\rf-rainfall-forecast\.virt\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


WunderGround: Weather Features (Holt-Winters)

In [ ]:
feature_wg = ["Temperature", "DewPoint", "Humidity", "Visibility", "WindSpeed", "Pressure"]
for feature in feature_wg:
    result = seasonal_decompose(df[feature], model='additive', period=365)
    result.plot()
    plot_acf(df[feature])
    plot_pacf(df[feature])
    plt.show()

In [46]:
df.head()

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,Visibility,WindSpeed,Pressure,Latitude,Longitude,Elevation,Wind,Condition
DateTime,,,,,,,,,,,,,,,
2002-08-01,0.5,27.72,26.91,0.81,83.500000,73.500000,73.208333,6.000000,3.625000,29.63250,4.847301,115.406703,6.5,E,Mostly Cloudy
2002-08-02,46.0,27.72,26.91,0.81,82.666667,75.416667,80.375000,5.583333,6.666667,29.65250,4.847301,115.406703,6.5,NE,Mostly Cloudy
2002-08-03,23.0,27.72,26.91,0.81,81.958333,74.250000,78.833333,5.916667,6.041667,29.67875,4.847301,115.406703,6.5,SSE,Mostly Cloudy
2002-08-04,9.5,27.72,26.91,0.81,82.791667,73.000000,73.458333,5.916667,5.750000,29.67625,4.847301,115.406703,6.5,WNW,Mostly Cloudy
2002-08-05,1.5,27.72,26.91,0.81,81.708333,74.125000,79.500000,5.083333,6.083333,29.69750,4.847301,115.406703,6.5,NW,Mostly Cloudy


In [47]:
df_wg = df.iloc[:, 4:10]
df_wg

,Temperature,DewPoint,Humidity,Visibility,WindSpeed,Pressure
DateTime,,,,,,
2002-08-01,83.500000,73.500000,73.208333,6.000000,3.625000,29.632500
2002-08-02,82.666667,75.416667,80.375000,5.583333,6.666667,29.652500
2002-08-03,81.958333,74.250000,78.833333,5.916667,6.041667,29.678750
2002-08-04,82.791667,73.000000,73.458333,5.916667,5.750000,29.676250
2002-08-05,81.708333,74.125000,79.500000,5.083333,6.083333,29.697500
...,...,...,...,...,...,...
2021-01-30,81.041667,74.250000,81.125000,6.000000,6.583333,29.695000
2021-01-31,81.750000,74.500000,80.041667,6.000000,5.166667,29.701250
2021-02-01,81.333333,75.166667,82.708333,5.875000,7.583333,29.732500


In [ ]:
def apply_holt_winters(data, feature, seasonal_periods=365):
    print(f"\nProcessing feature: {feature}")
    
    # Split data into train (80%) and test (20%)
    train_size = int(len(data) * 0.7)
    train = data[:train_size]
    test = data[train_size:]
    
    # Fit Holt-Winters model
    model = ExponentialSmoothing(
        train, 
        trend='add',       # Change to 'mul' if necessary
        seasonal='add',    # Change to 'mul' if necessary
        seasonal_periods=seasonal_periods
    ).fit()
    
    # Forecast the next values
    predictions = model.forecast(steps=len(test))
    
    # Evaluate the model
    mae = mean_absolute_error(test, predictions)
    rmse = np.sqrt(mean_squared_error(test, predictions))
    print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}")
    
    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(train, label="Training Data")
    plt.plot(test, label="Actual Values")
    plt.plot(predictions, label="Predictions", linestyle="-")
    plt.legend()
    plt.title(f"Holt-Winters Forecast for {feature}")
    plt.show()

# Apply Holt-Winters to each column in df_wg
for col in df_wg.columns:
    apply_holt_winters(df_wg[col], feature=col, seasonal_periods=365)